## FBA simulation
Simulating the FBA part of the model with `cobrapy` to check that the FBA part is working.

In [4]:
from __future__ import print_function, division
import pandas as pd
import cobra
from sbmlutils import fbc
from pprint import pprint

In [7]:
# load model
source = './results/diauxic_fba.xml'
mfba = cobra.io.read_sbml_model(source)
cobra.io.sbml3.validate_sbml_model(source)

(<Model diauxic_fba at 0x7f68c89f7950>,
 {'SBML errors': [], 'other': [], 'validator': [], 'warnings': []})

In [8]:
# objective function & boundaries
pprint(mfba.objective)
fbc.cobra_reaction_info(mfba)

{<Reaction v1 at 0x7f68c89f7810>: 1.0,
 <Reaction v2 at 0x7f68c89f7850>: 1.0,
 <Reaction v3 at 0x7f68c89f7890>: 1.0,
 <Reaction v4 at 0x7f68c89f78d0>: 1.0}


,lb,ub,objective
vO2,-1000,1000,NaN
vGlcxt,-1000,1000,NaN
vAc,-1000,1000,NaN
vX,0,1000,NaN
v1,0,1000,1
v2,0,1000,1
v3,0,1000,1
v4,0,1000,1


In [9]:
# simulate
# s = mfba.optimize()
s = mfba.optimize(objective_sense="maximize")

print(s)
print(mfba.solution.status)

<Solution 91.33 at 0x7f68c8a2a6d0>
optimal


In [10]:
# summary of solution
mfba.summary()

IN FLUXES     OUT FLUXES    OBJECTIVES
------------  ------------  ------------
Glcxt  1e+03  Ac  169       v1   0
O2     1e+03  X    91.3     v2  77.4
                            v3   0
                            v4  13.9


In [11]:
# FVA
# running flux variability analysis with the model
fva_result = cobra.flux_analysis.flux_variability_analysis(mfba, mfba.reactions)
pd.DataFrame.from_dict(fva_result).T.round(5)

,maximum,minimum
v1,0.00000,0.00000
v2,77.39938,77.39938
v3,0.00000,0.00000
v4,13.92625,13.92625
vAc,-168.78619,-168.78619
vGlcxt,1000.00000,1000.00000
vO2,1000.00000,1000.00000
vX,91.32563,91.32563


In [ ]:
from cobra.test import test_all
